

common_voice
https://huggingface.co/datasets/common_voice

JSUT (Japanese speech corpus of Saruwatari-lab., University of Tokyo)
https://sites.google.com/site/shinnosuketakamichi/publication/jsut

CSS10: A Collection of Single Speaker Speech Datasets for 10 Languages
https://github.com/Kyubyong/css10


TEDxJP-10K ASR Evaluation Dataset
https://github.com/laboroai/TEDxJP-10K

JVS (Japanese versatile speech) corpus
https://sites.google.com/site/shinnosuketakamichi/research-topics/jvs_corpus



In [ ]:
#download common voice
from datasets import load_dataset
common_voice_train = load_dataset("common_voice", "ja", split="train+validation")
common_voice_test = load_dataset("common_voice", "ja", split="test")


In [ ]:
#download JSUT (Japanese speech corpus of Saruwatari-lab., University of Tokyo)
!wget http://ss-takashi.sakura.ne.jp/corpus/jsut_ver1.1.zip
!unzip "./jsut_ver1.1.zip"

In [ ]:
# CSS10: A Collection of Single Speaker Speech Datasets for 10 Languages
# get KAGGLE_KEY from https://www.kaggle.com/<username>/account

import os
os.environ['KAGGLE_USERNAME'] = ""
os.environ['KAGGLE_KEY'] = ""
!pip install --user kaggle
!kaggle datasets download -d bryanpark/japanese-single-speaker-speech-dataset
!unzip japanese-single-speaker-speech-dataset.zip

In [ ]:
#download TEDxJP-10K ASR Evaluation Dataset git
!pip install youtube-dl
!git clone https://github.com/laboroai/TEDxJP-10K.git 
!sudo apt install sox


In [ ]:
#download TEDxJP-10K ASR Evaluation Dataset youtube
import os
from tqdm import tqdm
import youtube_dl
import time

lines=[]
with open("TEDxJP-10K/data/tedx-jp_urls.txt") as f:
    lines = f.readlines()
    
command="""youtube-dl \
	--extract-audio \
	--audio-format wav \
	--write-sub \
	--sub-format vtt \
	--sub-lang ja \
	--output "TEDxJP-10K/temp/raw/%(id)s.%(ext)s" \
    """
path="TEDxJP-10K/temp/raw/"

for url in tqdm(lines):
    urlId=url[32:-1]
    if os.path.exists(path+urlId+".ja.vtt") and os.path.exists(path+urlId+".wav"):
        continue
    print(urlId)
    stream = os.popen(command+lines[0])
    output = stream.read()
    time.sleep(10)


In [ ]:
#process TEDxJP-10K ASR Evaluation Dataset caption
!cd TEDxJP-10K&&python compose_tedxjp10k.py temp/raw


In [ ]:
#slice TEDxJP-10K audio based on time given
import torchaudio
import os 
datasets_TEDxJP = load_dataset('csv', data_files="TEDxJP-10K/TEDxJP-10K_v1.1/text",delimiter=" ",column_names=["name","sentence"])["train"]
saveDir="./TEDxJP/"
csvPath="TEDxJP.txt"

if not os.path.exists(saveDir):
    os.makedirs(saveDir)

if os.path.exists(csvPath):
    os.remove(csvPath)
    

def splitTime(batch):
    #save split audio
    splitResult=batch["name"].split("-")
    batch["path"]="TEDxJP-10K/TEDxJP-10K_v1.1/wav/"+"-".join(splitResult[:-2])+".16k.wav"
    batch["timeStart"]=int(int(splitResult[-2])/100*16000)
    batch["timeEnd"]=int((int(splitResult[-1])-int(splitResult[-2]))/100*16000)
    savePath=saveDir+batch["name"]+".wav"
    waveform, sample_rate = torchaudio.load(batch["path"], frame_offset=batch["timeStart"], num_frames=batch["timeEnd"])
    torchaudio.save(savePath, waveform, sample_rate)

    #save csv
    with open(csvPath, "a") as file:
        file.write(savePath+"|"+batch["sentence"]+"\n")
    return batch

datasets_TEDxJP=datasets_TEDxJP.map(splitTime)
print(datasets_TEDxJP)
print(datasets_TEDxJP[0])



In [7]:
!pip install gdown

# https://sites.google.com/site/shinnosuketakamichi/research-topics/jvs_corpus
# jvs
!gdown https://drive.google.com/uc?id=19oAw8wWn3Y7z6CKChRdAyGOB9yupL_Xt
!unzip jvs_ver1.zip






Downloading...
From: https://drive.google.com/uc?id=19oAw8wWn3Y7z6CKChRdAyGOB9yupL_Xt
To: /home/user/code/jupyter/voice/jvs_ver1.zip
100%|██████████████████████████████████████| 3.54G/3.54G [03:18<00:00, 17.8MB/s]


In [ ]:
#https://sites.google.com/site/shinnosuketakamichi/research-topics/jsss_corpus
# !gdown https://drive.google.com/uc?id=1NyiZCXkYTdYBNtD1B-IMAYCVa-0SQsKX
# !unzip jsss_ver1.zip


# import torchaudio
# from datasets import load_dataset
# from pathlib import Path  
# import os
# import glob

# savePath="./preprocess_jsss_ver1/"
# prepreocessCsvPath="preprocess_jsss_ver1.txt"
# if not os.path.exists(savePath):
#     os.makedirs(savePath)

# if os.path.exists(prepreocessCsvPath):
#     os.remove(prepreocessCsvPath)

    
# transcript_utf8_list=glob.glob("jsss_ver1/long-form/*/transcript_utf8/*.txt")
# transcript_utf8_list+=glob.glob("jsss_ver1/summarization/transcript_utf8/*.txt")


# for csvPath in transcript_utf8_list:
#     datasets = load_dataset('csv', data_files=csvPath,delimiter="\t",column_names=["timeStart","timeEnd","sentence"])["train"]    
    
#     fileName=Path(csvPath).stem
#     wavPath=os.path.abspath(os.path.join(csvPath, '../../wav24kHz16bit/')+fileName+".wav")
    
    
#     def splitTime(batch):
#         #save split audio
#         waveform, sample_rate = torchaudio.load(wavPath)
#         frameStart=int(float(batch["timeStart"])*sample_rate)
#         frameDuration=int((float(batch["timeEnd"])-float(batch["timeStart"]))*sample_rate)
#         savePathWav=savePath+fileName+"_"+str(frameStart)+".wav"
#         waveform, sample_rate = torchaudio.load(wavPath, frame_offset=frameStart, num_frames=frameDuration)
#         torchaudio.save(savePathWav, waveform, sample_rate)
        
#         #save csv
#         with open(prepreocessCsvPath, "a") as file:
#             file.write(savePathWav+"\t"+batch["sentence"]+"\n")
#         return batch

#     datasets=datasets.map(splitTime)
    

# #csvPath="preprocess_jsss_ver1.txt"
# #datasets = load_dataset('csv', data_files=csvPath,delimiter="\t",column_names=["path","sentence"])["train"]    


# import os
# from datasets import load_dataset
# transcript_utf8_list=glob.glob("jsss_ver1/short-form/*/transcript_utf8.txt")
# transcript_utf8_list+=["jsss_ver1/simplification/transcript_utf8.txt"]
# transcript_utf8_list

# datasets_jsss_list=[]
# for csvPath in transcript_utf8_list:
#     datasets = load_dataset('csv', data_files=csvPath,delimiter=":",column_names=["path","sentence"])["train"]
#     datasets = datasets.map(lambda batch: {'path': os.path.dirname(csvPath)+"/wav24kHz16bit/"+batch['path']+".wav"})
#     datasets_jsss_list+=[datasets]
# datasets_jsss_list